# Canal Model

The `Canal` class represents a canal with a trapezoidal cross-section. It uses Manning's equation to model the water flow. This notebook demonstrates how to use the `Canal` class to simulate the water dynamics in a canal segment.

## State Variables

The state of the canal is defined by the following variables:

- `volume` (float): The current volume of water in the canal (m^3).
- `water_level` (float): The current water level in the canal (m).
- `outflow` (float): The calculated outflow from the canal for the current step (m^3/s).

## Parameters

The physical characteristics of the canal are defined by these parameters:

- `bottom_width` (float): The width of the bottom of the canal (m).
- `length` (float): The length of the canal segment (m).
- `slope` (float): The longitudinal slope of the canal bed (dimensionless).
- `side_slope_z` (float): The slope of the canal sides (z in z:1, horizontal:vertical).
- `manning_n` (float): Manning's roughness coefficient.

## Simulation Example

The following code sets up and runs a simulation of a canal. It initializes the canal with a starting volume and physical parameters, then applies a constant inflow over a period of time. The results, including volume, water level, and outflow, are plotted.

**Note:** To run this notebook, ensure you have `matplotlib` installed (`pip install matplotlib`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.canal import Canal
from swp.core.interfaces import State

# Define the initial state and parameters for the canal
initial_state = State(volume=1000.0, water_level=0.0, outflow=0.0)
parameters = {
    'bottom_width': 10.0,    # meters
    'length': 1000.0,        # meters
    'slope': 0.001,          # dimensionless
    'side_slope_z': 2.0,     # z:1, horizontal:vertical
    'manning_n': 0.03        # Manning's roughness coefficient
}

# Create an instance of the Canal model
canal = Canal(name="main_canal", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 60  # time step in seconds (1 minute)
simulation_duration = 3600 * 2  # 2 hours
num_steps = int(simulation_duration / dt)
inflow_rate = 5.0  # constant inflow in m^3/s

# Store results for plotting
history = []

# Run the simulation loop
for _ in range(num_steps):
    canal.set_inflow(inflow_rate)
    current_state = canal.step(action=None, dt=dt)
    history.append(current_state.copy()) # Store a copy of the state at each step

# Print the final state of the canal
print("Final State:")
print(canal.get_state())

# Prepare data for plotting
time_in_hours = [i * dt / 3600 for i in range(num_steps)]
volumes = [s['volume'] for s in history]
water_levels = [s['water_level'] for s in history]
outflows = [s['outflow'] for s in history]

# Plot the results
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

ax1.plot(time_in_hours, volumes, label='Volume')
ax1.set_ylabel('Volume (m^3)')
ax1.set_title('Canal Dynamics Simulation')
ax1.grid(True)
ax1.legend()

ax2.plot(time_in_hours, water_levels, label='Water Level', color='orange')
ax2.set_ylabel('Water Level (m)')
ax2.grid(True)
ax2.legend()

ax3.plot(time_in_hours, outflows, label='Outflow', color='green')
ax3.set_xlabel('Time (hours)')
ax3.set_ylabel('Outflow (m^3/s)')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()
